In [ ]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

import mshr
import fenics as fe

In [4]:
def get_length(mesh):

  x_coords = mesh.coordinates()[:, 0]
  length = x_coords.max() - x_coords.min()

  return length

In [5]:
def define_boundaries(mesh):

    length = get_length(mesh)

    class Inflow(fe.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and fe.near(x[0], 0)

    class Walls(fe.SubDomain):
        def inside(self, x, on_boundary):
          return on_boundary and x[0] > 0 and x[0] < length

    class Outflow(fe.SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and fe.near(x[0], length)

    boundaries = fe.MeshFunction('size_t', mesh, mesh.topology().dim() - 1)
    boundaries.set_all(0)

    inflow_domain = Inflow()
    walls_domain = Walls()
    outflow_domain = Outflow()

    inflow_domain.mark(boundaries, 1)   # Mark inflow with '1'
    walls_domain.mark(boundaries, 2)    # Mark walls with '2'
    outflow_domain.mark(boundaries, 3)  # Mark outflow with '3'

    return boundaries